In [7]:
from ultralytics import YOLO


In [8]:
model = YOLO(r"C:\Users\Palak\Downloads\yolov8 trained model\besttraffic.pt")


In [9]:
model1 = YOLO(r"C:\Users\Palak\Downloads\yolov8 trained model\bestaccident.pt")

In [ ]:
pip install ultralytics opencv-python plyer


In [11]:
import smtplib


In [ ]:
# Import necessary libraries
import cv2
from ultralytics import YOLO
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import threading

# Load the YOLO model
model = YOLO('yolov8m(2).pt')  # Ensure this is your trained model with 2 classes: vehicle, accident

# Define the email sending function
def send_email(subject, body, to_email):
    sender_email = "itsnotpalak@gmail.com"
    app_password = "ninb kurp mted hadb"  # Use your app password
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = to_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, to_email, message.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Email error: {e}")
    finally:
        server.quit()

# Function to detect objects in video
def detect_in_video(video_path, frame_skip=5, vehicle_threshold=10):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    if not cap.isOpened():
        print("Error opening video file.")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip frames for efficiency

        # Resize frame for consistent processing
        frame = cv2.resize(frame, (640, 360))

        # Perform detection
        results = model(frame)
        vehicle_count = 0
        accident_detected = False

        # Iterate through detections
        for box in results[0].boxes:
            cls_id = int(box.cls[0])  # Get the class ID
            label = model.names[cls_id]  # Map the ID to the class label
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            color = (0, 255, 0)  # Default: Green for vehicles

            # Check the class and increment counters
            if label.lower() == "vehicle":
                vehicle_count += 1
            elif label.lower() == "accident":
                accident_detected = True
                color = (0, 0, 255)  # Red for accidents

            # Draw bounding boxes on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Display vehicle count and accident status on the frame
        cv2.putText(frame, f"Vehicles: {vehicle_count}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        status = "Traffic Detected" if vehicle_count > vehicle_threshold else "No Traffic"
        cv2.putText(frame, status, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255) if vehicle_count > vehicle_threshold else (0, 255, 0), 2)

        accident_status = "Accident Detected" if accident_detected else "No Accident"
        cv2.putText(frame, accident_status, (20, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255) if accident_detected else (0, 255, 0), 2)

        # Send email notification in a separate thread if a condition is met
        if vehicle_count > vehicle_threshold or accident_detected:
            threading.Thread(target=send_email, args=(
                "Traffic or Accident Alert",
                "Accident or Traffic Detected. Drive Safely ! ",
                "itsnotpalak@gmail.com",
            )).start()

        # Display the video frame
        cv2.imshow('Traffic and Accident Detection', frame)

        # Quit video with 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Test the function with your video
detect_in_video('cr.mp4', frame_skip=5)



0: 384x640 8 vehicles, 266.9ms
Speed: 13.6ms preprocess, 266.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 vehicles, 285.7ms
Speed: 0.0ms preprocess, 285.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 vehicles, 234.6ms
Speed: 2.8ms preprocess, 234.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 vehicles, 217.4ms
Speed: 1.0ms preprocess, 217.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 accident, 5 vehicles, 211.9ms
Speed: 0.0ms preprocess, 211.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 accident, 5 vehicles, 212.7ms
Speed: 0.0ms preprocess, 212.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 accident, 5 vehicles, 195.8ms
Speed: 0.0ms preprocess, 195.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 accident, 4 vehicles, 214.3ms
Speed: 0.

KeyboardInterrupt: 

Email sent successfully!
Email sent successfully!
